In [21]:
import pandas as pd
import numpy as np
from datetime import datetime
data = pd.read_csv('bus.csv')

In [93]:
def format_time(time):
    return datetime.fromisoformat(time[:-1])

In [94]:
def subtract_time(observed, scheduled):
    return (observed - scheduled).total_seconds() / 60

In [101]:
missing = data.id.count() - data.observed_visit_time.count() #missing data

observed_times = data.dropna(subset = ["observed_visit_time", "scheduled_visit_time"]) #dropping rows where observed_visit_time data is missing
observed_times_copy = observed_times.copy()
observed_times_copy['observed_visit_time'] = observed_times_copy['observed_visit_time'].map(format_time)
observed_times_copy['scheduled_visit_time'] = observed_times_copy['scheduled_visit_time'].map(format_time)

observed_times_formatted = observed_times_copy['observed_visit_time']
observed_times_formatted = observed_times_formatted.reset_index(drop=True)
scheduled_times_formatted = observed_times_copy['scheduled_visit_time']
scheduled_times_formatted = scheduled_times_formatted.reset_index(drop=True)

difference_times = np.empty(0)

for number in range(0, observed_times_formatted.shape[0] - 1):
    time = subtract_time(observed_times_formatted[number], scheduled_times_formatted[number])
    print(time)
    print(number)
    difference_times = np.append(difference_times, time)

print(difference_times)


4.5528
0
4.889733333333334
1
5.51615
2
4.8512
3
3.9713166666666666
4
4.555916666666667
5
4.554066666666667
6
4.47395
7
4.58985
8
4.830699999999999
9
5.130066666666666
10
5.4237
11
5.286983333333334
12
5.3533833333333325
13
5.418333333333334
14
5.339383333333333
15
5.184183333333333
16
4.8716
17
4.4702
18
4.299916666666666
19
4.103
20
4.087933333333334
21
22.183333333333334
22
30.988999999999997
23
4.1443666666666665
24
4.056783333333334
25
30.25115
26
29.611683333333335
27
3.7466166666666667
28
29.430733333333333
29
3.7188666666666665
30
28.629450000000002
31
3.5055166666666664
32
27.632133333333336
33
3.4010499999999997
34
3.4093500000000003
35
3.936
36
26.470916666666668
37
3.86575
38
25.800549999999998
39
3.649083333333333
40
4.0124
41
24.573933333333333
42
3.6212999999999997
43
23.527283333333333
44
23.102149999999998
45
3.299783333333333
46
22.726266666666668
47
3.226716666666667
48
21.744916666666665
49
3.4385
50
20.780116666666668
51
4.530183333333333
52
20.387716666666666
53
4.